In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
import sklearn

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=False)


f:\Projects\Coding-RasPi\HOML\homl-venv\Lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
X, y = mnist.data, mnist.target
X_train, X_test, y_train, y_test =  X[:60000], X[60000:], y[:60000], y[60000:]

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
cross_val_score(knn_clf, X_train, y_train, cv=3, scoring="accuracy")


array([0.9676 , 0.9671 , 0.96755])

In [4]:
from sklearn.model_selection import GridSearchCV

parameters = {'weights':('uniform', 'distance' ), 'n_neighbors':[3,5,7,9]}
knn = KNeighborsClassifier()
knn_gsv = GridSearchCV(knn, parameters, cv=3, scoring='accuracy')
knn_gsv.fit(X, y)
knn_gsv.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [5]:
grid_cv_res = pd.DataFrame(knn_gsv.cv_results_)
grid_cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
grid_cv_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,0.220538,0.001414,4.952027,0.043782,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.971029,0.969057,0.969571,0.969886,0.000836,1
0,0.216704,0.003065,5.558299,0.177139,3,uniform,"{'n_neighbors': 3, 'weights': 'uniform'}",0.969529,0.968114,0.968500,0.968714,0.000598,2
3,0.217371,0.002095,4.921022,0.028679,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.969572,0.967042,0.969057,0.968557,0.001092,3
5,0.223705,0.005202,5.166731,0.197096,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.969101,0.966357,0.967985,0.967814,0.001127,4
2,0.220205,0.007308,5.294420,0.013768,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.968372,0.966185,0.967814,0.967457,0.000928,5
4,0.218706,0.005202,5.325925,0.019831,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.967901,0.965242,0.967000,0.966714,0.001104,6
7,0.215371,0.001434,5.055379,0.001472,9,distance,"{'n_neighbors': 9, 'weights': 'distance'}",0.967258,0.964471,0.967428,0.966386,0.001356,7
6,0.211704,0.003172,5.392104,0.047359,9,uniform,"{'n_neighbors': 9, 'weights': 'uniform'}",0.966058,0.962885,0.966142,0.965029,0.001516,8


In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'weights': ('uniform', 'distance'), 'n_neighbors':randint(low=1, high=20)}
knn_random_search  = RandomizedSearchCV(knn, param_distributions=param_distribs, n_iter=10, cv=3, scoring="accuracy", random_state=42)
knn_random_search.fit(X_train, y_train)
knn_random_search_res = pd.DataFrame(knn_random_search.cv_results_)
knn_random_search_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
knn_random_search_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
6,0.186867,0.007599,3.631964,0.022400,4,distance,"{'n_neighbors': 4, 'weights': 'distance'}",0.97090,0.96980,0.97035,0.970350,0.000449,1
7,0.191867,0.003424,3.782157,0.095017,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.97040,0.96890,0.96870,0.969333,0.000759,2
8,0.213870,0.000472,4.047870,0.220286,2,distance,"{'n_neighbors': 2, 'weights': 'distance'}",0.96875,0.96685,0.96705,0.967550,0.000852,3
0,0.195868,0.012347,4.082209,0.298957,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.96700,0.96560,0.96740,0.966667,0.000772,4
3,0.189867,0.004289,3.638798,0.028668,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.96700,0.96560,0.96740,0.966667,0.000772,4
2,0.181532,0.002678,4.018198,0.101341,8,uniform,"{'n_neighbors': 8, 'weights': 'uniform'}",0.96365,0.96370,0.96570,0.964350,0.000955,6
9,0.197369,0.005808,3.932683,0.075227,12,distance,"{'n_neighbors': 12, 'weights': 'distance'}",0.96215,0.96290,0.96515,0.963400,0.001275,7
5,0.181198,0.002096,3.914181,0.032625,11,uniform,"{'n_neighbors': 11, 'weights': 'uniform'}",0.96110,0.96140,0.96365,0.962050,0.001138,8
1,0.193534,0.008379,4.352756,0.081676,15,uniform,"{'n_neighbors': 15, 'weights': 'uniform'}",0.95990,0.95910,0.96175,0.960250,0.001110,9
4,0.186366,0.004837,3.967356,0.026431,19,uniform,"{'n_neighbors': 19, 'weights': 'uniform'}",0.95810,0.95655,0.95955,0.958067,0.001225,10


In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, precision_recall_curve 
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
pd.options.display.max_rows = 4000

best_model = knn_random_search.best_estimator_ 
knn_predictions = best_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_f1 = f1_score(y_test, knn_predictions, average='micro')
knn_precision = precision_score(y_test, knn_predictions, average='micro')
knn_recall = recall_score(y_test, knn_predictions, average='micro')
knn_cm = confusion_matrix(y_test, knn_predictions)
knn_class_report = classification_report(y_test, knn_predictions, output_dict=False, target_names=['0','1','2','3','4','5','6','7','8','9'])

print(f"Accuracy is: {knn_accuracy},\nPrecision is {knn_precision},\nRecall is {knn_recall},\nf1-score is {knn_f1}\n")
print(f"Confussion Matrix: \n\n {knn_cm} \n")
print(f"Classification Report: \n\n {knn_class_report} \n")
# knn_fpr, knn_tpr, knn_thresholds = roc_curve_score(y_test, knn_predictions, pos_label='label')
#precisions, recalls, thresholds = precision_recall_curve(y_test, knn_predictions)
# plt.plot(knn_fpr, knn_tpr, linewidth=2, label='roc_curve')
# plt.show()

Accuracy is: 0.9714,
Precision is 0.9714,
Recall is 0.9714,
f1-score is 0.9714

Confussion Matrix: 

 [[ 973    1    1    0    0    1    3    1    0    0]
 [   0 1132    2    0    0    0    1    0    0    0]
 [  10    5  995    2    1    0    0   16    3    0]
 [   0    1    3  974    1   14    1    7    4    5]
 [   1    5    0    0  950    0    4    3    0   19]
 [   4    0    0    9    2  862    7    1    3    4]
 [   4    2    0    0    3    3  946    0    0    0]
 [   0   17    4    0    3    0    0  994    0   10]
 [   5    2    4   14    5   11    4    4  920    5]
 [   3    4    2    7    9    4    1   10    1  968]] 

Classification Report: 

               precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.97      1.00      0.98      1135
           2       0.98      0.96      0.97      1032
           3       0.97      0.96      0.97      1010
           4       0.98      0.97      0.97       982
           5  